In [ ]:
import os
from uuid import uuid1

import ipywidgets as widgets
from IPython.display import display
from ipycanvas import Canvas
from ipywidgets import Image


In [ ]:
img_list = []
for root, dirs, files in os.walk("img", topdown=False):
    for name in files:
        img_list.append(root + "/" + name)

In [ ]:
counter = 0

btn_next = widgets.Button(
    description="Next"
)

btn_skip = widgets.Button(
    description="Skip"
)

out = widgets.Output()

In [ ]:
DATASET_DIR = 'dataset_xy'
DATASET_DIR_UNKNOWN = 'dataset_unknown'

try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created because they already exist')

try:
    os.makedirs(DATASET_DIR_UNKNOWN)
except FileExistsError:
    print('Directories not created because they already exist')

In [ ]:
canvas = Canvas(width=224, height=224)

In [ ]:
x, y = None, None
image_path = None


@out.capture()
def on_canvas_click(x_in, y_in):
    global x, y
    x = int(x_in)
    y = int(y_in)
    print(f"{x} {y}")
    canvas.restore()
    canvas.stroke_style = "#00ff00"
    canvas.stroke_circle(x, y, 8)


def skip_snapshot():
    if image_path is not None:
        uuid = '%s' % (uuid1())
        new_image_path = os.path.join(DATASET_DIR_UNKNOWN, uuid + '.jpg')
        os.rename(image_path, new_image_path)


def save_snapshot():
    if image_path is not None:
        uuid = 'xy_%03d_%03d_%s' % (x, y, uuid1())
        new_image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
        os.rename(image_path, new_image_path)


def next_image():
    global image_path, counter, x, y
    canvas.clear()
    out.clear_output()

    if len(img_list) - 1 < counter:
        btn_next.disabled = True
        btn_skip.disabled = True
    else:
        image_path = img_list[counter]
        image = Image.from_file(image_path)

        canvas.draw_image(image, width=224, height=224)
        canvas.save()
        counter += 1

        x, y = None, None


def on_click_skip(b=None):
    skip_snapshot()
    next_image()


def on_click_next(b=None):
    if x is not None and y is not None:
        save_snapshot()
        next_image()


btn_skip.on_click(on_click_skip)
btn_next.on_click(on_click_next)
canvas.on_mouse_down(on_canvas_click)

next_image()

In [ ]:
data_collection_widget = widgets.VBox([
    widgets.HBox([canvas]),
    widgets.HBox([btn_skip, btn_next]),
])
display(data_collection_widget)
display(out)